# Домашняя работа

Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [25]:
from pathlib import Path
import json
from functools import reduce
from joblib import Parallel, delayed

In [26]:
%%time
n, mean, M2 = 0, 0.0, 0
for path in Path("les/imdb-user-reviews").glob("**/*"):
    if path.is_file() and path.suffix == ".json":
        with open(path, "r") as f:
            info = json.load(f)
        score = float(info["movieIMDbRating"])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)

print(mean, (M2 / n) ** (1 / 2))

8.03 1.0517128885774865
CPU times: total: 15.6 ms
Wall time: 9 ms


На основе этого кода соберите mapper и reducer:

In [43]:
def mapper(path):
    # Ваш код
    with open(path, "r") as f:
        info = json.load(f)
    score = float(info["movieIMDbRating"])
    return 1, score, 0.0


def reducer(score_data1, score_data2):
    #  Ваш код
    n1, mean1, M2_1 = score_data1
    n2, mean2, M2_2 = score_data2

    n = n1 + n2
    if n == 0:
        return 0, 0.0, 0.0

    delta = mean2 - mean1
    # delta = (mean2 * n2 - mean1 * n1) / n
    mean = mean1 + delta * n2 / n
    M2 = fsum([M2_1, M2_2, delta ** 2 * n1 * n2 / n])
    return n, mean, M2

In [44]:
%%time
n, mean, M2 = reduce(reducer, map(mapper, Path("les/imdb-user-reviews").rglob("*.json")))
print(mean, (M2 / n) ** (1 / 2))

8.03 1.0517128885774865
CPU times: total: 46.9 ms
Wall time: 7 ms
